<a href="https://colab.research.google.com/github/KaushalNaresh/Speech_enhancement_using_convolutional_autoencoders/blob/main/Source_code/Benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install SpeechRecognition
!pip install jiwer

In [ ]:
import os
import speech_recognition as spreg # converting audio to text
from scipy.io.wavfile import write
import numpy as np
from scipy.io import wavfile
import jiwer
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def test_function3(y):

    amp_max = np.iinfo(np.int16).max

    audio_combined = y  # newest new Nov 26

    audio_combined = scaling(audio_combined)

    data = audio_combined * amp_max / 4

    write("audio_combined.wav", sample_rate, data.astype(np.int16))
    # playsound("audio_combined.wav")

    # playsound("audio_combined.wav")

    output = audio_to_text("audio_combined.wav")
    # print(output)
    
    return output

In [ ]:
def audio_to_text(sound_file):
    recog = spreg.Recognizer()
    with spreg.AudioFile(sound_file) as source:
        speech = recog.record(source)  # use record instead of listning
        try:
            text = recog.recognize_google(speech)
            # print("The file contains: " + text)
        except spreg.UnknownValueError:
            # print("Unable to recognize the audio")
            text = "^"
        except spreg.RequestError as e:
            # print("Request error from Google Speech Recognition service; {}".format(e))
            text = "&"
    return text

In [ ]:
def scaling(audio_file):
    audio_file = audio_file - np.mean(audio_file)
    x_max = max(audio_file)

    output = audio_file / x_max
    return output

In [ ]:
# ToDo Here you need to pass the short cut link to the Folder Method_test_dataset on the Omdena google drive
dirname = '/content/drive/.shortcut-targets-by-id/1hUjbsv4_UZtKu-vtQdLU_wuMkY9kcb0g/MS-SNSD/Method_test_dataset'

In [ ]:
files = os.listdir(dirname + '/NoisySpeech_training')
os.listdir()

print(len(files))
files = sorted(files)
print(files)

3250
['noisy10_SNRdb_0.1_clnsp10.wav', 'noisy10_SNRdb_0.3040816326530612_clnsp10.wav', 'noisy10_SNRdb_0.5081632653061224_clnsp10.wav', 'noisy10_SNRdb_0.7122448979591837_clnsp10.wav', 'noisy10_SNRdb_0.9163265306122449_clnsp10.wav', 'noisy10_SNRdb_1.1204081632653062_clnsp10.wav', 'noisy10_SNRdb_1.3244897959183675_clnsp10.wav', 'noisy10_SNRdb_1.5285714285714287_clnsp10.wav', 'noisy10_SNRdb_1.73265306122449_clnsp10.wav', 'noisy10_SNRdb_1.9367346938775512_clnsp10.wav', 'noisy10_SNRdb_10.1_clnsp10.wav', 'noisy10_SNRdb_2.1408163265306124_clnsp10.wav', 'noisy10_SNRdb_2.344897959183674_clnsp10.wav', 'noisy10_SNRdb_2.548979591836735_clnsp10.wav', 'noisy10_SNRdb_2.753061224489796_clnsp10.wav', 'noisy10_SNRdb_2.9571428571428573_clnsp10.wav', 'noisy10_SNRdb_3.1612244897959187_clnsp10.wav', 'noisy10_SNRdb_3.3653061224489798_clnsp10.wav', 'noisy10_SNRdb_3.5693877551020408_clnsp10.wav', 'noisy10_SNRdb_3.773469387755102_clnsp10.wav', 'noisy10_SNRdb_3.9775510204081637_clnsp10.wav', 'noisy10_SNRdb_4.1816

In [ ]:
# Loading the Transcripts
transcript_dataframe = pd.read_excel(dirname + '/CleanSpeech_transcript.xlsx', header=None)
transcript_dataframe.columns = ['file_name', 'transcript']
transcript_dataframe.set_index('file_name', inplace=True)
transcript_dataframe['transcript'] = transcript_dataframe['transcript'].str.split().str.join(' ')

True

In [ ]:
# Settign WER Transformations
transformation = jiwer.Compose([
    jiwer.ToLowerCase(),
    jiwer.RemoveMultipleSpaces(),
    jiwer.ExpandCommonEnglishContractions(),
    jiwer.ExpandCommonEnglishContractions(),
    jiwer.ReduceToListOfListOfWords(word_delimiter=" ")
]) 

In [ ]:
sample_rate = 16000
y  = files
results = []
for name in tqdm(files):
  
  snr = round(float(name.split('_')[2]), 1)
  clean_file_name = name.split('_')[3]
  noisy_file_name = name.split('_')[0]
  if transcript_dataframe.loc[clean_file_name, 'transcript'] != 'exclude':
    dirname1 = dirname + '/NoisySpeech_training' + '/' + name


    # The script is loading the noisy Audio file
    Fs1, y_text = wavfile.read(dirname1)

    # Now you need to insert your preproccessing here before the test function is called



    # Calculating WER
    test_result = test_function3(y_text)
    clear_speech = transcript_dataframe.loc[clean_file_name, 'transcript']
    wer = jiwer.wer(clear_speech, 
                    test_result, 
                    truth_transform=transformation, 
                    hypothesis_transform=transformation)

    results.append([name, noisy_file_name, clean_file_name, snr, wer, clear_speech, test_result])



100%|██████████| 3250/3250 [50:55<00:00,  1.06it/s]


In [ ]:
data_result = pd.DataFrame(results)
data_result.columns = ['file_name', 'noisy_file_name', 'clean_file_name', 'SNR_level', 'WER', 'clean_text', 'audio_text']

In [ ]:
data_result.head()

,file_name,noisy_file_name,clean_file_name,SNR_level,WER,clean_text,audio_text
0,noisy11_SNRdb_0.1_clnsp11.wav,noisy11,clnsp11.wav,0.1,0.866667,lady Thatcher was the only women with full mem...,child abuse
1,noisy11_SNRdb_0.3040816326530612_clnsp11.wav,noisy11,clnsp11.wav,0.3,0.866667,lady Thatcher was the only women with full mem...,child abuse
2,noisy11_SNRdb_0.5081632653061224_clnsp11.wav,noisy11,clnsp11.wav,0.5,0.866667,lady Thatcher was the only women with full mem...,child abuse
3,noisy11_SNRdb_0.7122448979591837_clnsp11.wav,noisy11,clnsp11.wav,0.7,0.866667,lady Thatcher was the only women with full mem...,child abuse
4,noisy11_SNRdb_0.9163265306122449_clnsp11.wav,noisy11,clnsp11.wav,0.9,0.600000,lady Thatcher was the only women with full mem...,the only one with a full membership child abuse


In [ ]:
data_result.describe()

,SNR_level,WER
count,2800.000000,2800.000000
mean,5.100000,0.392939
std,2.953492,0.296466
min,0.100000,0.000000
25%,2.500000,0.156883
50%,5.100000,0.307692
75%,7.700000,0.583333
max,10.100000,1.714286
